In [ ]:
%tensorflow_version 1.x

from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

from google.colab import drive
from math import ceil
import matplotlib.pyplot as plt
import datetime
import numpy as np
import os

# Experiment configuration
USING_SPLITS = False
current_folder = '/content/gdrive/My Drive/public/sediments/'

samples = 'splits' if USING_SPLITS else 'images'
drive.mount('/content/gdrive')
!cp "{current_folder}{samples}.tar.gz" /tmp
!tar xzvf /tmp/{samples}.tar.gz
BASE = '/content/{}/folds'.format(samples)
ALL_TRAIN_SET_PATH = '/content/splits/train' if USING_SPLITS else '/content/images/training_set'
TEST_SET_PATH = '/content/splits/test' if USING_SPLITS else '/content/images/test_set'
BASE_RESULTS = current_folder + 'results/' + samples

CLASSES = ['background', 'chambers', 'channels', 'packing voids', 'planes', 'vesicles', 'vughs']

results_path = BASE_RESULTS + '/MLP'
os.makedirs(results_path, exist_ok=True)

# Network parameters
NUM_FOLDS = 1
input_size = 224
BATCH_SIZE = 32
EPOCHS = 80

all_acc_histories = []
# For loop repeating the training for each fold: fine-tuning freezing the base model
for fold in range(1, NUM_FOLDS + 1):
    fold_path = BASE + '/fold_' + str(fold)
    train_path = fold_path + '/training_set'
    val_path = fold_path + '/validation_set'
    results_file = open(results_path + '/info.txt', 'w' if fold == 1 else 'a') 
    
    hidden_units = 1000
    num_labels = 7
    model = Sequential()
    model.add(Dense(hidden_units, input_dim=input_size*input_size*3))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(hidden_units))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(hidden_units))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(hidden_units))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(hidden_units))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_labels))
    # this is the output for one-hot vector
    model.add(Activation('softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    
    train_batches = ImageDataGenerator(
        rescale=1. / 255, rotation_range=30, vertical_flip=True, horizontal_flip=True, zoom_range=[0.8, 1.2]
        ).flow_from_directory(train_path, target_size=(input_size, input_size), batch_size=BATCH_SIZE, classes=CLASSES)
    val_batches = ImageDataGenerator(rescale=1. / 255).flow_from_directory(
        val_path, target_size=(input_size, input_size), batch_size=BATCH_SIZE,classes=CLASSES)
    
    STEPS_EPOCH_TRAIN=train_batches.n//train_batches.batch_size
    STEPS_EPOCH_VALID=val_batches.n//val_batches.batch_size
  
    for e in range(EPOCHS):
        print('Epoch {}'.format(e))
        step = 0
        for x_batch, y_batch in train_batches:
            x_batch = np.reshape(x_batch, [-1, input_size * input_size * 3])
            model.fit(x_batch, y_batch, verbose=0)
            step += 1
            if (step >= STEPS_EPOCH_TRAIN):
                break

      acc = []
      step = 0
      for x_valid, y_valid in val_batches:
         x_valid = np.reshape(x_valid, [-1, input_size * input_size * 3])
        scores = model.evaluate(x_valid, y_valid, batch_size=BATCH_SIZE, verbose=False)
        step += 1
        acc.append(scores[1])
        if (step >= STEPS_EPOCH_VALID):
          break

      print('val acc: {}'.format(sum(acc) / len(acc)))

test_batches = ImageDataGenerator(rescale=1. / 255).flow_from_directory(
    TEST_SET_PATH, target_size=(input_size, input_size), batch_size=BATCH_SIZE, classes=CLASSES)
STEPS_EPOCH_TEST = test_batches.n // test_batches.batch_size
acc = []
step = 0
for x_test, y_test in test_batches:
  x_test = np.reshape(x_test, [-1, input_size * input_size * 3])
  scores = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=False)
  step += 1
  print('Step {0}/{1}'.format(step, STEPS_EPOCH_TEST))
  acc.append(scores[1])
  if (step >= STEPS_EPOCH_TEST):
    break

model.summary()